# Atomic tensor models

``graph-pes`` provides models that target atomic "tensorial" properties, ranging from atomic energies and charges, dipoles, NMR anisotropic parameters, to higher rank tensors.

## Available models

The currently available models extend the ``MACE`` and ``NequIP`` architectures:
1. [``TensorMACE``](https://jla-gardner.github.io/graph-pes/models/many-body/tensormace.html)

1. [``ZEmbeddingTensorMACE``](https://jla-gardner.github.io/graph-pes/models/many-body/tensormace.html#graph_pes.models.ZEmbeddingTensorMACE)

1. [``TensorNequIP``](https://jla-gardner.github.io/graph-pes/models/many-body/tensornequip.html)

1. [``ZEmbeddingTensorNequIP``](https://jla-gardner.github.io/graph-pes/models/many-body/tensornequip.html#graph_pes.models.ZEmbeddingTensorNequIP)

Both NequIP- and MACE-based models implement two learning approaches: ``direct`` and ``tensor_product``. To learn tensor components with nonstandard spherical harmonics, such as `0o; 1e; 2o; 3e;...`,  using a MACE-based model, we recommend the ``tensor_product`` approach.


## Data preparation

For the remainder of this notebook, we will reconstruct lightweight ``TensorNequIP`` NMR models targeting the magnetic shielding tensor (MS) used for amorphous silica in [this paper](https://doi.org/10.1063/5.0274240).

First we download the training data:

In [1]:
!wget https://github.com/cbenmahm/anistropic-nmr-parameters-data/raw/refs/heads/main/data/train_test/train.xyz

--2025-11-24 19:24:58--  https://github.com/cbenmahm/anistropic-nmr-parameters-data/raw/refs/heads/main/data/train_test/train.xyz
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/cbenmahm/anistropic-nmr-parameters-data/refs/heads/main/data/train_test/train.xyz [following]
--2025-11-24 19:24:58--  https://media.githubusercontent.com/media/cbenmahm/anistropic-nmr-parameters-data/refs/heads/main/data/train_test/train.xyz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52559049 (50M) [application/octet-stream]
Saving to: ‘train.xyz’

train.xyz           100%[===================>]  5

In [ ]:
import load_atoms

structures = load_atoms.load_dataset("./train.xyz")

We need to transform the outputs to spherical tensor and decompose them into a basis of spherical harmonics so they can learnt by the tensor models

In [25]:
import torch
from e3nn import io

ct_symm = io.CartesianTensor("ij=ji")
ct_antisymm = io.CartesianTensor("ij=-ji")

/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  w

In [26]:
for frm in structures:
    t_ms = torch.from_numpy(frm.arrays["ms"].reshape(-1, 3, 3))
    symm = ct_symm.from_cartesian(t_ms)
    anti = ct_antisymm.from_cartesian(t_ms)

    ms = torch.cat((symm[..., :1], anti, symm[..., 1:]), dim=-1)
    ms = ms.numpy()
    frm.arrays["ms_SH"] = ms

/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/Users/work/source/miniconda3/envs/uv/lib/python3.11/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  w

In [27]:
import ase.io

train, val, test = structures.random_split([0.8, 0.1, 0.1])

ase.io.write("train-nmr.xyz", train)
ase.io.write("val-nmr.xyz", val)
ase.io.write("test-nmr.xyz", test)

## Configuration file

Now that we've saved our labelled structures to suitable files, we're ready to train a model.

To do this, we have specified the following in the ``tensornequip-direct-sio2.yaml`` file:

* the model architecture to instantiate and train, here [TensorNequIP](https://jla-gardner.github.io/graph-pes/models/many-body/tensornequip.html). Note that we also include a [FixedTensorOffset](https://jla-gardner.github.io/graph-pes/models/offsets.html#graph_pes.models.FixedTensorOffset) component to account for the fact that the amophous silica labels have an arbitrary offset.
* the data to train on, here a random split of the [amorphous silica](https://github.com/cbenmahm/anistropic-nmr-parameters-data/raw/refs/heads/main/data/train_test/train.xyz) dataset we just downloaded
* the loss function to use, here a per-atom RMSE
* and various other training hyperparameters (e.g. the learning rate, batch size, etc.)

You can download [this config file](https://raw.githubusercontent.com/jla-gardner/graph-pes/refs/heads/main/docs/source/quickstart/tensornequip-direct-sio2.yaml) for the direct approach using wget:

In [28]:
%%bash

if [ ! -f tensornequip-direct-sio2.yaml ]; then
    wget https://raw.githubusercontent.com/jla-gardner/graph-pes/refs/heads/main/docs/source/quickstart/tensornequip-direct-sio2.yaml -O tensornequip-direct-sio2.yaml
fi

or this one for the tensor_product approach:

In [29]:
%%bash

if [ ! -f tensornequip-tp-sio2.yaml ]; then
    wget https://raw.githubusercontent.com/jla-gardner/graph-pes/refs/heads/main/docs/source/quickstart/tensornequip-direct-sio2.yaml -O tensornequip-direct-sio2.yaml
fi

## Training

The models are trained in the same way as the usual ``GraphPES`` models using the [graph-pes-train](https://jla-gardner.github.io/graph-pes/cli/graph-pes-train/root.html) command.

In [30]:
!graph-pes-train tensornequip-direct-sio2.yaml \
    general/run_id=train-nequip-tensor

[graph-pes INFO]: Started `graph-pes-train` at 2025-11-24 19:36:24.567
[graph-pes INFO]: Successfully parsed config.
[graph-pes INFO]: Logging to graph-pes-results/train-nequip-tensor/rank-0.log
[graph-pes INFO]: ID for this training run: train-nequip-tensor
[graph-pes INFO]: 
Output for this training run can be found at:
    └─ graph-pes-results/train-nequip-tensor
        ├─ rank-0.log         # find a verbose log here
        ├─ model.pt           # the best model (according to valid/loss/total)
        ├─ lammps_model.pt    # the best model deployed to LAMMPS
        ├─ train-config.yaml  # the complete config used for this run
        └─ summary.yaml       # the summary of the training run

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[graph-pes INFO]: Preparing data
[graph-pes INFO]: Caching neighbour lists for 680 structures with cutoff 5.5329999923706055, property mapping {'ms_SH': 'tensor'} and torch dtype t

## Model analysis

First, we load the model

In [ ]:
import torch

from graph_pes.models import load_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = (
    load_model(
        "graph-pes-results/train-nequip-tensor/model.pt"
    )  # load the model
    .to(device)  # move to GPU if available
    .eval()  # set to evaluation mode
)

ValueError: Expected the loaded object to be a GraphPESModel but got <class 'graph_pes.models.addition.TensorAdditionModel'>

In [32]:
from graph_pes.utils.calculator import GraphPESCalculator

calculator = GraphPESCalculator(best_model)

NameError: name 'best_model' is not defined

Then, we need to tranform the predictions back to Cartesian coordinates

In [33]:
for frm in test:
    calculator.calculate(frm, properties=["tensor"])
    tensor = calculator.results["tensor"]

    symm = torch.cat((tensor[..., :1], tensor[..., 4:]), dim=-1)
    tensor_symm = ct_symm.to_cartesian(symm)

    tensor_antisymm = ct_antisymm.to_cartesian(tensor[..., 1:4])

    tensor = tensor_symm + tensor_antisymm
    tensor = tensor.cpu().numpy()
    frm["ms_ML"] = tensor

NameError: name 'calculator' is not defined

and now you can use libraries like [``soprano``](https://github.com/CCP-NC/soprano) to exctract NMR tensor properties or [``MRSimulator``](https://github.com/deepanshs/mrsimulator) to simulate spectra!